<a href="https://colab.research.google.com/github/aiswarya-1422/AI-SMART-SEARCH-for-Kerala-GOVT-/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentence-transformers faiss-cpu
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 44.0 MB/s eta 0:00:00


In [ ]:
import zipfile
import os

zip_path = "/content/processed_json_en.zip"   # your uploaded file
extract_path = "/content/processed_json_en"   # folder to extract into

# Unzip the JSONs
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Extracted files to:", extract_path)
print("Files found:", len(os.listdir(extract_path)))


✅ Extracted files to: /content/processed_json_en
Files found: 1


In [ ]:
import json
import os

# Load saved FAISS index and embeddings
index = faiss.read_index("/content/semantic_index.faiss")
embeddings = np.load("/content/doc_embeddings.npy")

# Rebuild your documents list from JSON folder
folder_path = "/content/processed_json_en"
# Adjusting the path to look inside the nested folder
nested_folder_path = os.path.join(folder_path, "processed_json_en")
json_files = [os.path.join(nested_folder_path, f) for f in os.listdir(nested_folder_path) if f.endswith(".json")]

documents = []
for file in json_files:
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        # Extract text from either "summary" or "content"
        text = data.get("summary") or data.get("content") or ""
        # If text is a list, join it into a single string
        if isinstance(text, list):
            text = " ".join(text)

        if text.strip():
            documents.append(text)

print(f"✅ Reloaded {len(documents)} summarized documents from {nested_folder_path}.")
print("Files in folder:", os.listdir(nested_folder_path))

✅ Reloaded 451 summarized documents from /content/processed_json_en/processed_json_en.
Files in folder: ['G.O.P.30-05.json', 'GO(P) No 317-13-Fin  dated 04-07-2013.json', 'Circular no 05-2014-Fin  dated 01-02-2014.json', 'CircularNo70-2021-FinDated02-09-2021_61.json', 'CircularNo56-2022-FinDated11-07-2022_61.json', 'CircularNo93-2022-FinDated14-11-2022_61.json', 'GO.P.630-10-Fin.json', 'Notification -SS-1-142-2021Dated-27-05-2021_95.json', 'GO(P)No93-2020-FinDated08-07-2020_94.json', '488-Notification_95.json', 'GO(Rt) No 1589-2017-Fin  dated 27-02-2017.json', 'G.O.P.303-07.json', 'G.O.P No.503-08-Fin.json', '4-2019-FinDated14-01-2019_61.json', 'GO(P) No 409-2014-Fin  dated 23-09-2014.json', 'CircularNo95-2022-FinDated16-11-2022_61.json', 'SS-1-186-2025_95.json', 'Notification No 28886-SS1-2014-Fin Dated 21-03-2014.json', 'CircularNo72-2022-FinDated19-08-2022_61.json', 'GO(Rt)No6226-2025-FinDated28-07-2025_12.json', 'Notification No 6214-ADmn A6-2012-Fin Dated 02-06-2012.json', 'GO.P.3

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

def semantic_search(query, top_k=3):
    query_vector = model.encode([query])
    D, I = index.search(query_vector, top_k)
    return [documents[i] for i in I[0]]


In [ ]:
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")


Device set to use cpu


In [ ]:
def ask_question(query, top_k=3):

    top_docs = semantic_search(query, top_k=top_k)
    context = " ".join(top_docs)

    answer = qa_pipeline(question=query, context=context)
    return answer['answer'], answer['score']

question = "What is the main focus of the budget document?"
answer, score = ask_question(question)
print(f" Question: {question}\n💬 Answer: {answer}\n Confidence: {score:.4f}")


 Question: What is the main focus of the budget document?
💬 Answer: developmental schemes and related expenditure
 Confidence: 0.1123


In [ ]:
while True:
    q = input("\nAsk a question (or type 'exit'): ")
    if q.lower() == "exit":
        break
    answer, score = ask_question(q)
    print(f"💬 Answer: {answer}\n🔢 Confidence: {score:.4f}")



Ask a question (or type 'exit'): what is the main focus of circular documents
💬 Answer: register all temporary employees
🔢 Confidence: 0.4329

Ask a question (or type 'exit'): exit


In [ ]:
import gradio as gr

def chatbot(query):
    answer, score = ask_question(query)
    return f"Answer: {answer}\nConfidence: {score:.4f}"

gr.Interface(fn=chatbot,
             inputs="text",
             outputs="text",
             title="Smart PDF Assistant",
             description="Ask questions about summarized Kerala Finance documents").launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fa9021b071caf511ad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
